In [1]:
# Dataset download from https://www.cs.cmu.edu/~jwieting/

with open("/root/projects/NLP_projects/synpg/para-nmt-5m-processed.txt") as file:
    lines = file.readlines()

In [2]:
raw_sentence = list()
for idx, sentence in enumerate(lines):
    raw_sentence.append(lines[idx].split("\t")[0])

In [1]:
import nltk
import benepar, spacy

benepar.download('benepar_en3')
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

def parsing_tree(sentence):

    doc = nlp(sentence)
    sent = list(doc.sents)[0]
    tree = sent._.parse_string

    result = "(ROOT " + tree + ")"

    return result

2023-03-29 23:01:25.778817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 23:01:28.060889: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-29 23:01:28.061052: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-29 23:01:28.061060: W tensorfl

In [2]:
from apply_bpe import BPE, read_vocabulary

# load bpe codes
import codecs
bpe_codes = codecs.open("/root/projects/NLP_projects/synpg/bpe.codes", encoding='utf-8')
bpe_vocab = codecs.open("/root/projects/NLP_projects/synpg/vocab.txt", encoding='utf-8')

def bpe_tokenize(sentence):
    bpe_codes = codecs.open("/root/projects/NLP_projects/synpg/bpe.codes", encoding='utf-8')
    bpe_vocab = codecs.open("/root/projects/NLP_projects/synpg/vocab.txt", encoding='utf-8')
    bpe_vocab = read_vocabulary(bpe_vocab, 50)
    bpe = BPE(bpe_codes, '@@', bpe_vocab, None)

    result = bpe.segment(sentence).split()

    return result

In [3]:
def append_sos_eos(sentence):
    sentence.insert(0, "<sos>")
    sentence.insert(len(sentence), "<eos>")
    return sentence

def is_paren(tok):
    return tok == ")" or tok == "("

def deleaf(tree):
    nonleaves = ''
    for w in str(tree).replace('\n', '').split():
        w = w.replace('(', '( ').replace(')', ' )')
        nonleaves += w + ' '

    arr = nonleaves.split()
    for n, i in enumerate(arr):
        if n + 1 < len(arr):
            tok1 = arr[n]
            tok2 = arr[n + 1]
            if not is_paren(tok1) and not is_paren(tok2):
                arr[n + 1] = ""

    nonleaves = " ".join(arr)
    return nonleaves.split()

from nltk import ParentedTree

def Parsertokenize(synt_):
    synt_ = ParentedTree.fromstring(synt_)
    synt_ = deleaf(synt_)
    synt_ = [f'<{w}>' for w in synt_]
    return synt_

In [4]:
from apply_bpe import BPE, read_vocabulary

# load bpe codes
import codecs
bpe_codes = codecs.open("/root/projects/NLP_projects/synpg/bpe.codes", encoding='utf-8')
bpe_vocab = codecs.open("/root/projects/NLP_projects/synpg/vocab.txt", encoding='utf-8')

def bpe_tokenize(sentence):
    bpe_codes = codecs.open("/root/projects/NLP_projects/synpg/bpe.codes", encoding='utf-8')
    bpe_vocab = codecs.open("/root/projects/NLP_projects/synpg/vocab.txt", encoding='utf-8')
    bpe_vocab = read_vocabulary(bpe_vocab, 50)
    bpe = BPE(bpe_codes, '@@', bpe_vocab, None)

    result = bpe.segment(sentence).split()

    return result


In [58]:
# Create dataloader for dataloader
from tqdm import tqdm_notebook

# Cut data to 1000 sentence
data = raw_sentence[0:1000]

train_data = list()

for idx in tqdm_notebook(range(len(data))):
#for idx in tqdm_notebook(range(50)):
    input  = bpe_tokenize(data[idx])
    tree   = Parsertokenize(parsing_tree(data[idx]))
    target = append_sos_eos(input.copy())

    train_data.append([input, tree, target])

/tmp/ipykernel_408/3445887119.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(data))):


  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/distributions/distribution.py:45: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [1]:
import pickle
from utils import synt2str, sent2str, load_embedding
from apply_bpe import BPE, read_vocabulary

#with open("train_data.pkl", "wb") as file:
# file = open("train_data.pkl", "wb")
# pickle.dump(train_data, file)
file = open("train_data.pkl", "rb")
train_data = pickle.load(file)

with open("dictionary.pkl", "rb") as file:
    syndict = pickle.load(file)

vocab_dict = syndict.word2idx

In [2]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('spacy',language='en_core_web_sm')
text_pipeline = lambda x: [vocab_dict[x_] if x_ in vocab_dict else vocab_dict["<unk>"] for x_ in x ]

2023-03-30 00:13:44.740969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-30 00:13:46.587772: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-30 00:13:46.587934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-30 00:13:46.587941: W tensorfl

In [3]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch

pad_idx = vocab_dict['<pad>'] ##get the pad index from the vocab

def collate_batch(batch):
    sent_list, synt_lst, trg_list = [], [], []
    # print(len(batch))
    # sens_, syns_, trgs_ = batch
    # for sen_, syn_, trg_ in zip(sens_, syns_, trgs_):
    for sen_, syn_, trg_ in batch:
        processed_sent = torch.tensor(text_pipeline(sen_), dtype=torch.int64)
        sent_list.append(processed_sent)
        processed_synt = torch.tensor(text_pipeline(syn_), dtype=torch.int64)
        synt_lst.append(processed_synt)
        processed_trg = torch.tensor(text_pipeline(trg_), dtype=torch.int64)
        trg_list.append(processed_trg)

    return pad_sequence(sent_list, padding_value=pad_idx, batch_first=True), pad_sequence(synt_lst, padding_value=pad_idx, batch_first=True), pad_sequence(trg_list, padding_value=pad_idx, batch_first=True)

In [4]:
from torch.utils.data import DataLoader

train_range = int(len(train_data) * 0.7)

train_set = train_data[0:train_range]
val_set   = train_data[train_range:]
# test_set = train_data[90:]

train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(val_set, batch_size=32, shuffle=True, collate_fn=collate_batch)
#test_dataloader = DataLoader(test_set, batch_size=4, shuffle=False, collate_fn=collate_batch)

In [5]:
import torch, torchdata, torchtext
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
import random, math, time
from torch.autograd import Variable
import operator
import numpy as np

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 6969
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cuda:0


In [6]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout = 0.1, max_len = 5000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-np.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding):
        # Residual connection + pos encoding
        token_embedding = token_embedding + self.pos_encoding[:token_embedding.size(0), :]
        return self.dropout(token_embedding)

In [7]:
class seq2seqTransformer(nn.Module):
    def __init__(self, input_dim, emb_dim, device, word_dropout = 0.4, dropout = 0.1):
        super(seq2seqTransformer, self).__init__()
        self.input_dim = input_dim 
        self.emb_dim = emb_dim 
        self.word_dropout = word_dropout
        self.dropout = dropout
        self.device = device
        
        self.scale = np.sqrt(self.emb_dim)
        # self.scale = torch.sqrt(torch.IntTensor([self.hid_dim])).to(device)

        # vcocabulary embedding
        self.embedding_encoder = nn.Embedding(input_dim, emb_dim)
        self.embedding_decoder = nn.Embedding(input_dim, emb_dim)
        # positional encoding
        self.pos_encoder = PositionalEncoding(emb_dim, dropout = 0.0)

        self.transformer = nn.Transformer(d_model = emb_dim, nhead = 12, dropout = dropout)
        # linear Transformation
        self.linear = nn.Linear(emb_dim, input_dim)
        self.init_weights()

    def load_embedding(self, embedding):  #synPG applied with GloVe glove.840B.300d.txt
        self.embedding_encoder.weight.data.copy_(torch.from_numpy(embedding)) 
        self.embedding_decoder.weight.data.copy_(torch.from_numpy(embedding))  

    def init_weights(self):
        initrange = 0.1
        # initialize cocabulary matrix weight
        self.embedding_encoder.weight.data.uniform_(-initrange, initrange)
        self.embedding_decoder.weight.data.uniform_(-initrange, initrange)
        # initialize linear weight
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.fill_(0)

    def generate_square_mask(self, max_sent_len, max_synt_len):
        size = max_sent_len + max_synt_len + 2 #<sos> and <eos>
        mask = torch.zeros((size, size))
        mask[:max_sent_len, max_sent_len:] = float("-inf")
        mask[max_sent_len:, :max_sent_len] = float("-inf")
        return mask
        
    def forward(self, sents, synts, trg):
        #sents  : batch_size, seq_len
        #synts  : batch_size, seq_len
        #trgs   : batch_size, seq_len 
        batch_size   = sents.size(0)
        max_sent_len = sents.size(1)
        max_synt_len = synts.size(1) - 2    # count without <sos> and <eos>
        max_targ_len = trg.size(1) - 2      # count without <sos> and <eos>

        # apply word dropout
        drop_mask = torch.bernoulli(self.word_dropout * torch.ones(max_sent_len)).bool().to(self.device)
        #sents = sents.masked_fill(drop_mask, -1e10)
        sents = sents.masked_fill(drop_mask, 0)

        #print(sents.shape)
        #print(self.embedding_encoder(sents).shape)

        # sentence, syntax => embedding
        sent_embeddings = self.embedding_encoder(sents).transpose(0, 1) * self.scale
        #sent_emb = [seq_len, batch size, emb_size]
        synt_embeddings = self.embedding_encoder(synts).transpose(0, 1) * self.scale
        synt_embeddings = self.pos_encoder(synt_embeddings) 
        #synt_emb = [seq_len, batch size, emb_size]
        en_embeddings = torch.cat((sent_embeddings, synt_embeddings), dim=0)
        #en_emb = [seq_len, batch size, emb_size*2]

        # do not allow cross attetion
        src_mask = self.generate_square_mask(max_sent_len, max_synt_len).to(self.device)
        
        # target => embedding
        de_embeddings = self.embedding_decoder(trg[:, :-1]).transpose(0, 1) * self.scale
        de_embeddings = self.pos_encoder(de_embeddings)
        
        # sequential mask
        trg_mask = self.transformer.generate_square_subsequent_mask(max_targ_len+1).to(self.device)
        
        # forward
        outputs = self.transformer(en_embeddings, de_embeddings, src_mask=src_mask, tgt_mask=trg_mask)
        
        # apply linear layer to vocabulary size
        outputs = outputs.transpose(0, 1)
        outputs = self.linear(outputs.contiguous().view(-1, self.emb_dim))
        outputs = outputs.view(batch_size, max_targ_len + 1, self.input_dim)
        #output = [batch size, trg_len, vocab_size]
        return outputs
    
    def generate(self, sents, synts, max_len = 30, sample=True, temp=0.5):
        #sents  : batch_size, seq_len
        #synts  : batch_size, seq_len
        batch_size   = sents.size(0)
        max_sent_len = sents.size(1)
        max_synt_len = synts.size(1) - 2  # count without <sos> and <eos>
        max_targ_len = max_len
        
        # output index starts with <sos>
        idxs = torch.zeros((batch_size, max_targ_len+2), dtype=torch.long).to(self.device)
        idxs[:, 0] = 1
        
        # sentence, syntax => embedding
        sent_embeddings = self.embedding_encoder(sents).transpose(0, 1) * self.scale
        synt_embeddings = self.embedding_encoder(synts).transpose(0, 1) * self.scale
        synt_embeddings = self.pos_encoder(synt_embeddings)
        en_embeddings = torch.cat((sent_embeddings, synt_embeddings), dim=0)
        
        # do not allow cross attetion
        src_mask = self.generate_square_mask(max_sent_len, max_synt_len).to(self.device)
        
        # starting index => embedding
        de_embeddings = self.embedding_decoder(idxs[:, :1]).transpose(0, 1) * self.scale
        de_embeddings = self.pos_encoder(de_embeddings)
        
        # sequential mask
        trg_mask = self.transformer.generate_square_subsequent_mask(de_embeddings.size(0)).to(self.device)
        
        # encode
        memory = self.transformer.encoder(en_embeddings, mask=src_mask)
        
        # auto-regressively generate output
        for i in range(1, max_targ_len+2):
            if i % 5 == 0:
                print(f'epoch : {i}')
            # decode
            outputs = self.transformer.decoder(de_embeddings, memory, tgt_mask=trg_mask)
            outputs = self.linear(outputs[-1].contiguous().view(-1, self.emb_dim))
            
            # get argmax index or sample index
            if not sample:
                values, idx = torch.max(outputs, 1)
            else:
                probs = F.softmax(outputs/temp, dim=1)
                idx = torch.multinomial(probs, 1).squeeze(1)
            
            # save to output index
            idxs[:, i] = idx
            
            # concatenate index to decoding
            de_embeddings = self.embedding_decoder(idxs[:, :i+1]).transpose(0, 1) * self.scale
            de_embeddings = self.pos_encoder(de_embeddings)
            
            # new sequential mask
            trg_mask = self.transformer.generate_square_subsequent_mask(de_embeddings.size(0)).to(self.device)
        
        return idxs[:, 1:]

In [8]:
from gensim.test.utils import datapath

#you have to put this file in some python/gensim directory; just run it and it will inform where to put....
glove_file = datapath('/root/projects/NLP_projects/synpg/glove.6B.300d.txt')

In [9]:

#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = "cpu"
print(device)

from utils import load_embedding

input_dim   = len(vocab_dict)
emb_dim     = 300  #fasttext
word_dropout = 0.4 #following SynPG
dropout      = 0.1

embedding = load_embedding(glove_file, syndict)

model = seq2seqTransformer(input_dim=input_dim, emb_dim = emb_dim, device=device, word_dropout = word_dropout, dropout = dropout)
model = model.to(device)
model.load_embedding(embedding)
# model.embedding_encoder.weight.data = fast_embedding #apply fasttext instead of Glove 840b 300d.txt (5.56 GB) TT
# model.embedding_decoder.weight.data = fast_embedding

cpu
load 22696 of 31414 from pretrained word embeddings



In [10]:
def train(model, loader, optimizer, criterion, clip, loader_length):
    
    model.train()
    epoch_loss = 0
    for sents_, synts_, trgs_ in loader:

        batch_size   = sents_.size(0)
        max_sent_len = sents_.size(1)
        max_synt_len = synts_.size(1) - 2  # count without <sos> and <eos>
        
        optimizer.zero_grad()

        # Put input into device
        sents_ = sents_.to(device)
        synts_ = synts_.to(device)
        trgs_ = trgs_.to(device)
        
        #forward 
        outputs = model(sents_, synts_, trgs_)

        # calculate loss
        targs_ = trgs_[:, 1:].contiguous().view(-1) #Without <SOS>
        outputs_ = outputs.contiguous().view(-1, outputs.size(-1))
        
        optimizer.zero_grad()

        loss = criterion(outputs_, targs_)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / loader_length

def evaluate(model, loader, criterion, loader_length):

    #turn off dropout (and batch norm if used)
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():

        for sents_, synts_, trgs_ in loader:

            batch_size   = sents_.size(0)
            max_sent_len = sents_.size(1)
            max_synt_len = synts_.size(1) - 2  # count without <sos> and <eos>
            
            #forward 
            outputs = model(sents_, synts_, trgs_)

            # calculate loss
            targs_ = trgs_[:, 1:].contiguous().view(-1) #Without <SOS>
            outputs_ = outputs.contiguous().view(-1, outputs.size(-1))
            
            loss = criterion(outputs_, targs_)
            
            epoch_loss += loss.item()
        
    return epoch_loss / loader_length

In [11]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [12]:
import torch.optim as optim

lr = 10e-4 #Following SynPG
wd = 10e-5 #Following SynPG
#training hyperparameters
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx) #combine softmax with cross entropy

In [13]:
train_loader_length = len(list(iter(train_dataloader)))
val_loader_length   = len(list(iter(valid_dataloader)))
#test_loader_length  = len(list(iter(test_dataloader)))
#train_loader_length, val_loader_length, test_loader_length

In [14]:
import time
import math
from tqdm import tqdm_notebook
best_valid_loss = float('inf')
num_epochs = 5
clip       = 1

save_path = '/root/projects/NLP_projects/synpg/models/prototype_synpg.pt'

train_losses = []
valid_losses = []

for epoch in tqdm_notebook(range(num_epochs)):
#for epoch in range(num_epochs):

    start_time = time.time()

    # training
    train_loss = train(model, train_dataloader, optimizer, criterion, clip, train_loader_length)
    valid_loss = evaluate(model, valid_dataloader, criterion, val_loader_length)
    
    #for plotting
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    # save model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), save_path)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    #lower perplexity is better

/tmp/ipykernel_351/4195108782.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(num_epochs)):


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 01 | Time: 6m 38s
	Train Loss: 7.797 | Train PPL: 2433.916
	 Val. Loss: 6.932 |  Val. PPL: 1024.307
Epoch: 02 | Time: 2m 8s
	Train Loss: 6.299 | Train PPL: 543.972
	 Val. Loss: 6.936 |  Val. PPL: 1028.517
Epoch: 03 | Time: 2m 4s
	Train Loss: 6.171 | Train PPL: 478.782
	 Val. Loss: 6.957 |  Val. PPL: 1050.232
Epoch: 04 | Time: 2m 17s
	Train Loss: 6.137 | Train PPL: 462.788
	 Val. Loss: 6.930 |  Val. PPL: 1022.896
Epoch: 05 | Time: 2m 19s
	Train Loss: 6.141 | Train PPL: 464.452
	 Val. Loss: 6.916 |  Val. PPL: 1008.502


In [50]:
for sents_, synts_, trgs_ in valid_dataloader:
    break

sents_ = sents_[0:4]
synts_ = synts_[0:4]
trgs_  = trgs_[0:4]

outputs = model(sents_, synts_, trgs_)

In [51]:
targs_ = trgs_[:, 1:].contiguous().view(-1) #Without <SOS>
outputs_ = outputs.contiguous().view(-1, outputs.size(-1))

In [ ]:
def generate(model, loader, criterion, loader_length,vocab_transform):
    #turn off dropout (and batch norm if used)
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():

        for sents_, synts_, trgs_ in loader:

            batch_size   = sents_.size(0)
            max_sent_len = sents_.size(1)
            max_synt_len = synts_.size(1) - 2  # count without <sos> and <eos>
            
            # generate
            idxs = model.generate(sents_, synts_, sents_.size(1), temp=0.5)
            
            # write output
            for sent, idx, synt in zip(sents_.cpu().numpy(), idxs.cpu().numpy(), synts_.cpu().numpy()):
                print(synt2str(synt[1:], vocab_transform)+'\n')
                print(sent2str(sent, vocab_transform)+'\n')
                print(synt2str(idx, vocab_transform)+'\n')
                print("--\n")